# convert and merge SVI

In [35]:
import geopandas
import pandas

In [36]:
data = geopandas.read_file('../datas/04-svi/query.geojson')
data.drop(columns=['geometry']).to_csv('../datas/04-svi/svi.csv', index=False)

In [37]:
csvdata = pandas.read_csv('../datas/04-svi/svi.csv')
csvdata.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,RPL_THEMES
0,36,NEW YORK,NY,Albany,36001000100,"Census Tract 1, Albany County, New York",0.8524
1,36,NEW YORK,NY,Albany,36001000200,"Census Tract 2, Albany County, New York",0.7603
2,36,NEW YORK,NY,Albany,36001000300,"Census Tract 3, Albany County, New York",0.8359
3,36,NEW YORK,NY,Albany,36001000401,"Census Tract 4.01, Albany County, New York",0.4764
4,36,NEW YORK,NY,Albany,36001000403,"Census Tract 4.03, Albany County, New York",0.3459


In [38]:
geodata = geopandas.read_file('../datas/00-tractTOzcta/Modified Zip Code Tabulation Areas (MODZCTA).geojson')
geodata.head()

,label,modzcta,pop_est,zcta,geometry
0,"10001, 10118",10001,23072,"10001, 10119, 10199","MULTIPOLYGON (((-73.98774 40.74407, -73.98819 ..."
1,10002,10002,74993,10002,"MULTIPOLYGON (((-73.99750 40.71407, -73.99709 ..."
2,10003,10003,54682,10003,"MULTIPOLYGON (((-73.98864 40.72293, -73.98876 ..."
3,10004,10004,3028,10004,"MULTIPOLYGON (((-74.00827 40.70772, -74.00937 ..."
4,10005,10005,8831,"10005, 10271","MULTIPOLYGON (((-74.00783 40.70309, -74.00786 ..."


In [39]:
txtdata = pandas.read_csv('../datas/00-tractTOzcta/zcta_tract_rel_10.txt')
txtdata.head()

,ZCTA5,STATE,COUNTY,TRACT,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,...,TRAREA,TRAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,TRPOPPCT,TRHUPCT,TRAREAPCT,TRAREALANDPCT
0,601,72,1,956300,72001956300,4271,1706,44663250,44572589,18570,...,44924558,44833897,23.00,22.03,26.67,26.74,98.5,98.33,99.42,99.42
1,601,72,1,956400,72001956400,2384,1037,32830481,32492074,18570,...,37782601,37191697,12.84,13.39,19.61,19.50,79.6,80.14,86.89,87.36
2,601,72,1,956500,72001956500,3126,1240,44969548,44809680,18570,...,44969548,44809680,16.83,16.01,26.85,26.89,100.0,100.00,100.00,100.00
3,601,72,1,956600,72001956600,2329,972,1981101,1981101,18570,...,1981101,1981101,12.54,12.55,1.18,1.19,100.0,100.00,100.00,100.00
4,601,72,1,956700,72001956700,2053,948,1380041,1380041,18570,...,1380041,1380041,11.06,12.24,0.82,0.83,100.0,100.00,100.00,100.00


In [41]:
txtdata = txtdata[['ZCTA5', 'GEOID']]
txtdata.head()

,ZCTA5,GEOID
0,601,72001956300
1,601,72001956400
2,601,72001956500
3,601,72001956600
4,601,72001956700


In [42]:
txtdata.dtypes

ZCTA5    int64
GEOID    int64
dtype: object

In [43]:
geodata.dtypes

label         object
modzcta       object
pop_est       object
zcta          object
geometry    geometry
dtype: object

In [44]:
tmp = {}
for index, row in geodata.iterrows():
    if row['modzcta'] != '99999':
        for zcta in row['zcta'].split(', '):
            tmp[zcta] = row['modzcta']
geodatacleaned = pandas.DataFrame(tmp.items(), columns=['ZCTA5', 'modzcta'])
geodatacleaned = geodatacleaned.astype({'ZCTA5': 'int64', 'modzcta': 'int64'})
geodatacleaned

,ZCTA5,modzcta
0,10001,10001
1,10119,10001
2,10199,10001
3,10002,10002
4,10003,10003
...,...,...
209,11691,11691
210,11692,11692
211,11693,11693
212,11694,11694


In [45]:
txt_geo = txtdata.join(geodatacleaned.set_index('ZCTA5'), on='ZCTA5').dropna(subset=None, inplace=False)
txt_geo = txt_geo.astype({'modzcta': 'int64'})
txt_geo

,ZCTA5,GEOID,modzcta
9554,10001,36061005800,10001
9555,10001,36061007600,10001
9556,10001,36061009100,10001
9557,10001,36061009300,10001
9558,10001,36061009500,10001
...,...,...,...
13406,11694,36081093800,11694
13407,11694,36081094201,11694
13408,11697,36081091601,11697
13409,11697,36081091602,11697


In [46]:
csvdata.dtypes

ST              int64
STATE          object
ST_ABBR        object
COUNTY         object
FIPS            int64
LOCATION       object
RPL_THEMES    float64
dtype: object

In [54]:
csv_txt_geo = csvdata.join(txt_geo.set_index('GEOID'), on='FIPS').dropna(subset=['ZCTA5', 'modzcta'], inplace=False).astype({'ZCTA5': 'int64', 'modzcta':'int64'})
csv_txt_geo = csv_txt_geo.reset_index(drop=True)
csv_txt_geo

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,RPL_THEMES,ZCTA5,modzcta
0,36,NEW YORK,NY,Bronx,36005000100,"Census Tract 1, Bronx County, New York",-999.0000,11370,11370
1,36,NEW YORK,NY,Bronx,36005000200,"Census Tract 2, Bronx County, New York",0.9544,10473,10473
2,36,NEW YORK,NY,Bronx,36005000400,"Census Tract 4, Bronx County, New York",0.6322,10473,10473
3,36,NEW YORK,NY,Bronx,36005001600,"Census Tract 16, Bronx County, New York",0.9761,10473,10473
4,36,NEW YORK,NY,Bronx,36005001900,"Census Tract 19, Bronx County, New York",0.9752,10454,10454
...,...,...,...,...,...,...,...,...,...
1358,36,NEW YORK,NY,Kings,36047123700,"Census Tract 1237, Kings County, New York",0.6982,11205,11205
1359,36,NEW YORK,NY,Kings,36047150200,"Census Tract 1502, Kings County, New York",0.2405,11215,11215
1360,36,NEW YORK,NY,Kings,36047150200,"Census Tract 1502, Kings County, New York",0.2405,11218,11218
1361,36,NEW YORK,NY,Kings,36047152200,"Census Tract 1522, Kings County, New York",0.4982,11218,11218


In [55]:
csv_txt_geo.to_csv('../datas/04-svi/svi-cleaned.csv', index=False)

In [57]:
test = geopandas.read_file('../datas/04-svi/querynew.geojson')
test.shape

(2000, 3)

In [58]:
countys = pandas.read_csv('../datas/0-InternetEducation.csv')
countys['County'].unique()

array(['Bronx County', 'Kings County', 'Nassau County', 'New York County',
       'Queens County', 'Richmond County'], dtype=object)

In [59]:
csvdata = pandas.read_csv('../datas/04-svi/cdcsvi.csv')
csvdata.shape

(5294, 25)

In [61]:
csvdata['County'].unique()

array(['Richmond', 'Ulster', 'New York', 'Hudson', 'Dutchess', 'Kings',
       'Westchester', 'Nassau', 'Queens', 'Rockland', 'Suffolk',
       'Sullivan', 'Putnam', 'Orange', 'Bronx', ' Bronx', 'Fairfield',
       'Litchfield', 'New Haven', 'Somerset', 'Ocean', 'Passaic',
       'Morris', 'Warren', 'Bergen', 'Mercer', 'Monmouth', 'Essex',
       'Hunterdon', 'Sussex', 'Union', 'Middlesex'], dtype=object)

In [65]:
csvdata = csvdata[csvdata['County'].isin(['Bronx', 'Kings', 'Nassau', 'New York', 'Queens', 'Richmond'])]
csvdata = csvdata[['NAMELSAD10','Pop_T','SV_Index2']]
csvdata

,NAMELSAD10,Pop_T,SV_Index2
0,Census Tract 17,1788,0.353482
1,Census Tract 6,2408,0.433675
2,Census Tract 7,4915,0.409195
3,Census Tract 18,731,0.374946
4,Census Tract 3,1927,0.390923
...,...,...,...
3297,Census Tract 185,7466,0.445247
3298,Census Tract 209,4361,0.469949
3299,Census Tract 222,3425,0.422964
3300,Census Tract 238,2615,0.377509


In [68]:
bronx = geopandas.read_file('../datas/04-svi/querybronx.geojson')
kings = geopandas.read_file('../datas/04-svi/querykings.geojson')
nassau = geopandas.read_file('../datas/04-svi/querynassau.geojson')
ny = geopandas.read_file('../datas/04-svi/queryny.geojson')
queens = geopandas.read_file('../datas/04-svi/queryqueens.geojson')
richmond = geopandas.read_file('../datas/04-svi/queryrichmond.geojson')

In [77]:
csvdatanew = pandas.concat([bronx, kings, nassau, ny, queens, richmond], ignore_index=True).drop(columns=['geometry'], inplace=False)
csvdatanew = csvdatanew.astype({'FIPS': 'int64'})
csvdatanew.to_csv('../datas/04-svi/gissvi-concat.csv', index=False)

In [85]:
csvdatanew

,FIPS,RPL_THEMES,COUNTY
0,36005000100,-999.0000,Bronx
1,36005000200,0.9544,Bronx
2,36005000400,0.6322,Bronx
3,36005001600,0.9761,Bronx
4,36005001900,0.9752,Bronx
...,...,...,...
2539,37153970900,0.6960,Richmond
2540,37153971000,0.9082,Richmond
2541,37153971100,0.9082,Richmond
2542,51159040100,0.6738,Richmond


In [74]:
txt_geo

,ZCTA5,GEOID,modzcta
9554,10001,36061005800,10001
9555,10001,36061007600,10001
9556,10001,36061009100,10001
9557,10001,36061009300,10001
9558,10001,36061009500,10001
...,...,...,...
13406,11694,36081093800,11694
13407,11694,36081094201,11694
13408,11697,36081091601,11697
13409,11697,36081091602,11697


In [78]:
csvdatanew.dtypes

FIPS            int64
RPL_THEMES    float64
COUNTY         object
dtype: object

In [83]:
csv_txt_geo = csvdatanew.join(txt_geo.set_index('GEOID'), on='FIPS').dropna(subset=['ZCTA5', 'modzcta', 'FIPS'], inplace=False).astype({'ZCTA5': 'int64', 'modzcta':'int64'})
csv_txt_geo = csv_txt_geo.reset_index(drop=True)
csv_txt_geo

,FIPS,RPL_THEMES,COUNTY,ZCTA5,modzcta
0,36005000100,-999.0000,Bronx,11370,11370
1,36005000200,0.9544,Bronx,10473,10473
2,36005000400,0.6322,Bronx,10473,10473
3,36005001600,0.9761,Bronx,10473,10473
4,36005001900,0.9752,Bronx,10454,10454
...,...,...,...,...,...
2889,36085030302,0.5113,Richmond,10303,10303
2890,36085030302,0.5113,Richmond,10314,10314
2891,36085031901,0.9625,Richmond,10303,10303
2892,36085031902,0.9183,Richmond,10303,10303


In [84]:
csv_txt_geo.dtypes

FIPS            int64
RPL_THEMES    float64
COUNTY         object
ZCTA5           int64
modzcta         int64
dtype: object

In [86]:
csv_txt_geo.to_csv('../datas/04-svi/svi-cleaned.csv', index=False)

In [88]:
csv_txt_geo = txt_geo.join(csvdatanew.set_index('FIPS'), on='GEOID').dropna(subset=['ZCTA5', 'modzcta'], inplace=False).astype({'ZCTA5': 'int64', 'modzcta':'int64'})
csv_txt_geo = csv_txt_geo.reset_index(drop=True)
csv_txt_geo

,ZCTA5,GEOID,modzcta,RPL_THEMES,COUNTY
0,10001,36061005800,10001,0.1429,New York
1,10001,36061007600,10001,0.2589,New York
2,10001,36061009100,10001,0.3534,New York
3,10001,36061009300,10001,0.5049,New York
4,10001,36061009500,10001,0.3847,New York
...,...,...,...,...,...
2889,11694,36081093800,11694,0.8598,Queens
2890,11694,36081094201,11694,0.7239,Queens
2891,11697,36081091601,11697,0.2466,Queens
2892,11697,36081091602,11697,-999.0000,Queens


In [89]:
len(txt_geo['GEOID'].unique())

2182

In [90]:
len(csvdatanew['FIPS'].unique())

2544

# merge data

In [91]:
import pandas

In [92]:
data = pandas.read_csv('../datas/04-svi/svi-cleaned.csv')
data.shape

(2894, 5)

In [93]:
data.head()

,FIPS,RPL_THEMES,COUNTY,ZCTA5,modzcta
0,36005000100,-999.0000,Bronx,11370,11370
1,36005000200,0.9544,Bronx,10473,10473
2,36005000400,0.6322,Bronx,10473,10473
3,36005001600,0.9761,Bronx,10473,10473
4,36005001900,0.9752,Bronx,10454,10454


In [94]:
data = data[['modzcta', 'RPL_THEMES']]
data.head()

,modzcta,RPL_THEMES
0,11370,-999.0000
1,10473,0.9544
2,10473,0.6322
3,10473,0.9761
4,10454,0.9752


In [95]:
data = data[data['RPL_THEMES'] != -999.0000]
data

,modzcta,RPL_THEMES
1,10473,0.9544
2,10473,0.6322
3,10473,0.9761
4,10454,0.9752
5,10455,0.9752
...,...,...
2889,10303,0.5113
2890,10314,0.5113
2891,10303,0.9625
2892,10303,0.9183


In [98]:
data[data['modzcta'] == 10473]

,modzcta,RPL_THEMES
1,10473,0.9544
2,10473,0.6322
3,10473,0.9761
6,10473,0.9710
8,10473,0.1060
12,10473,0.8002
19,10473,0.8665
23,10473,0.9496
27,10473,0.9890
57,10473,0.8427


In [103]:
merged = pandas.DataFrame(columns=['modzcta', 'avgsvi'])
merged

,modzcta,avgsvi


In [104]:
for mod in data['modzcta'].unique():
    tmpdict = {}
    tmpdata = data[data['modzcta'] == mod]
    tmpdict['modzcta'] = mod
    tmpdict['avgsvi'] = tmpdata['RPL_THEMES'].sum() / len(tmpdata)
    merged = merged.append(tmpdict, ignore_index=True)
merged = merged.astype({'modzcta': 'int64'})
merged

,modzcta,avgsvi
0,10473,0.821064
1,10454,0.975442
2,10455,0.961394
3,10472,0.956680
4,10451,0.938777
...,...,...
172,10312,0.233494
173,10314,0.410404
174,10307,0.215775
175,10309,0.208713


In [105]:
merged.to_csv('../datas/04-svi/svi-merged.csv', index=False)